Во-первых, как уже говорилось, процедура в PostgreSQL не может вернуть результирующий набор. Функция может вернуть результирующий набор. Фактически, функция может вернуть множество результирующих наборов. Кроме того, поскольку это PostgreSQL, эти результирующие наборы являются объектами; это означает, что вы можете вернуть не просто набор столбцов и строк (кортежи), но таблицу, курсор (хороший тип, и да, это хороший тип курсора, по крайней мере, в PostgreSQL. Как процедуры, так и функции допускают выходные параметры, которые называются параметрами INOUT.

Следующее большое отличие между процедурами и функциями состоит в том, что функции не могут управлять транзакциями.

In [5]:
%%sql
--reset the database
--connect first to postgres database

--DROP DATABASE IF EXISTS hamshackradio;
--CREATE DATABASE hamshackradio;

--connect to hamshackradio
--start with a radio schema
--may add more schema for more functionality in the future

CREATE SCHEMA radio;

GRANT USAGE ON SCHEMA radio TO postgres;

--radios is the root of the whole thing
CREATE TABLE IF NOT EXISTS radio.radios
	(radio_id int CONSTRAINT pkradios PRIMARY KEY GENERATED always AS IDENTITY,
	radio_name varchar(100) NOT NULL,
	manufacturer_id int NOT NULL,
	picture BYTEA NULL,
	connectortype_id int NOT NULL,
	digitalmode_id int NULL);

--natural key for radios
CREATE UNIQUE INDEX radios_radio_name_manufacturer_id_unique ON radio.radios USING BTREE(radio_name,manufacturer_id);

--a manufacturer makes multiple products
CREATE TABLE IF NOT EXISTS radio.manufacturers
	(manufacturer_id int CONSTRAINT pkmanufacturers PRIMARY KEY GENERATED always AS IDENTITY,
	manufacturer_name varchar(100) NOT NULL);

--natural key for manufacturers
CREATE UNIQUE INDEX manufacturers_manufacturer_name_unique ON radio.manufacturers USING BTREE(manufacturer_name);

--Antenna & radios have connectors
CREATE TABLE IF NOT EXISTS radio.connectortypes
(connectorytype_id int CONSTRAINT pkconnectortypes PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
connectortype_name varchar(100) NOT NULL);

--natural key for connectortypes
CREATE UNIQUE INDEX connectortypes_connectorytype_name ON radio.connectortypes USING BTREE(connectorytype_id);

--some radios support digital modes
CREATE TABLE IF NOT EXISTS radio.digitalmodes
(digitalmode_id int CONSTRAINT pkdigitalmode PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
digitalmode_name varchar(100) NOT NULL);

--natural key for digitalmodes
CREATE UNIQUE INDEX digitalmodes_digitalmode_name ON radio.digitalmodes USING BTREE(digitalmode_id);

--relate radios to various lookup tables
ALTER TABLE radio.radios
ADD foreign key (manufacturer_id) references radio.manufacturers;

ALTER TABLE radio.radios
ADD FOREIGN KEY (connectortype_id) REFERENCES radio.connectortypes;

ALTER TABLE radio.digitalmodes
ADD FOREIGN KEY (digitalmode_id) REFERENCES radio.digitalmodes;

--radios need antenna
CREATE TABLE IF NOT EXISTS radio.antenna
	(antenna_id int CONSTRAINT pkantenna PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
		antenna_name varchar(100) NOT NULL,
	 manufacturer_id int NOT NULL,
	 connectortype_id int NOT NULL
	);

--natural key for antenna
CREATE UNIQUE INDEX antenna_antenna_name_unique ON radio.antenna USING BTREE(antenna_name,manufacturer_id);

ALTER TABLE radio.antenna
ADD FOREIGN KEY (manufacturer_id) REFERENCES radio.manufacturers;

ALTER TABLE radio.antenna
ADD FOREIGN KEY (connectortype_id) REFERENCES radio.connectortypes;

--amateur radios work within bands, defined by country
CREATE TABLE IF NOT EXISTS radio.bands
(band_id int CONSTRAINT pkbands PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
 band_name varchar(100) NOT NULL,
 frequency_start_khz numeric(9,2) NOT NULL,
 frequency_end_khz numeric(9,2) NOT NULL,
 country_id int NOT NULL);

--natural key for bands
CREATE UNIQUE INDEX bands_band_name_country_id ON radio.bands USING BTREE(band_name,country_id);

--radios support multiple bands
CREATE TABLE IF NOT EXISTS radio.radiobands
(radio_id int NOT NULL,
band_id int NOT NULL,
CONSTRAINT pkradiobands PRIMARY KEY(radio_id,band_id));

ALTER TABLE radio.radiobands
ADD FOREIGN KEY (radio_id) REFERENCES radio.radios;

ALTER TABLE radio.radiobands
ADD FOREIGN KEY (band_id) REFERENCES radio.bands;

--antenna support multiple bands
CREATE TABLE IF NOT EXISTS radio.antennabands
(antenna_id int NOT NULL,
band_id int NOT NULL,
CONSTRAINT pkantennabands PRIMARY KEY(antenna_id,band_id));

ALTER TABLE radio.antennabands
ADD FOREIGN KEY (antenna_id) REFERENCES radio.antenna;

ALTER TABLE radio.antennabands
ADD FOREIGN KEY (band_id) REFERENCES radio.bands;

--country table
CREATE TABLE IF NOT EXISTS radio.countries
(country_id int CONSTRAINT pkcountry PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
country_name varchar(100) NOT NULL);

ALTER TABLE radio.bands
ADD FOREIGN KEY (country_id) REFERENCES radio.countries;


--which radio is using which antenna
CREATE TABLE IF NOT EXISTS radio.radioantenna
(radio_id int NOT NULL,
antenna_id int NOT NULL,
CONSTRAINT pkradioantenna PRIMARY KEY(radio_id,antenna_id));


--logging schema for storing points & other data types
CREATE SCHEMA logging;

GRANT USAGE ON SCHEMA logging TO postgres;

CREATE TABLE IF NOT EXISTS logging.logs
(log_id int CONSTRAINT pklogs PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
log_date timestamptz  NOT NULL,
log_callsign text,
log_location point NOT NULL);

CREATE TABLE IF NOT EXISTS logging.parksontheair
(parksontheair_id int CONSTRAINT pkparksontheair PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
contacts text[]);

--making sure I have this extension
CREATE EXTENSION btree_gist;








Exception: [42P06] ОШИБКА: схема "radio" уже существует

In [ ]:
%%sql

--sample data for the hamradio db
INSERT INTO radio.manufacturers
(manufacturer_name)
VALUES
('Yeasu'),('Icom'),('Baofeng'),('BTech'),('TidRadio'),('Xiegu'),('Raddiodity');

INSERT INTO radio.countries
(country_name)
VALUES
('USA'),('United Kingdom');

INSERT INTO radio.bands
(band_name,frequency_start_khz,frequency_end_khz,country_id)
VALUES
('2 Meters',144000.0,146000.0,1),
('70 cm',420000.0,450000.0,1),
('20 Meters',14000.0,14350.0,1),
('144MHz Band',144000.0,146000.0,2),
('432MHz Band',430000.0,440000.0,2),
('20 Metres',14000.0,14350.0,2);

INSERT INTO radio.connectortypes
(connectortype_name)
VALUES
('BNC'),('SMA Male'),('SMA Female'),('PL259');


INSERT INTO radio.antenna
(antenna_name,manufacturer_id,connectortype_id)
VALUES
('Rubber duck',3,2),
('Rubber duck',2,3),
('Multi-band',1,4);

INSERT INTO radio.antennabands
(band_id,antenna_id)
VALUES
(1,1),(2,1),(4,1),(5,1),
(1,2),(2,2),(4,2),(5,2),
(3,3),(5,3);

INSERT INTO radio.digitalmodes
(digitalmode_name)
VALUES
('FusionX'),('DMR');

INSERT INTO radio.radios
(radio_name,manufacturer_id,connectortype_id,digitalmode_id)
VALUES
('FT3D',2,2,1),('UV5R',3,3,NULL),('G90',6,4,NULL);

INSERT INTO radio.radioantenna
(radio_id,antenna_id)
VALUES
(1,2),(2,1),(3,3);


INSERT INTO logging.logs
(log_date,log_callsign,log_location)
VALUES
(CURRENT_TIMESTAMP,'KC1KCE','35.947, -96.149'),
(CURRENT_TIMESTAMP,'KC1KCE','35.963, -96.114'),
(CURRENT_TIMESTAMP,'KC1KCE','35.985, -96.106');

INSERT INTO logging.parksontheair
(contacts)
VALUES
(ARRAY['KC1KCE', 'KI5EGH', 'KJ7VSF','N4DAD','KD7IFN']),
(ARRAY['KN4XP','KA0GAV','NM8GS','W3DET']),
(ARRAY['W3DET','N4AUG','KC3WRX','WA9EBX','W1OW','K4RFH','W9DC','WJ8L','N4HES']);

INSERT INTO radio.radiobands
(radio_id, band_id)
VALUES
(1,1),(1,2),(1,4),(1,5),
(2,1),(2,2),
(3,3),(3,6);




In [ ]:
%%sql
CREATE OR REPLACE
FUNCTION radio.radiodetails (radioid INT)
RETURNS TABLE(radio_id int,
radio_name varchar(100))
AS $$
SELECT
	r.radio_id,
	r.radio_name
FROM
	radio.radios AS r
WHERE
	r.radio_id = $1 $$
LANGUAGE SQL;

In [2]:
%%sql
select * from radio.radios

,radio_id,radio_name,manufacturer_id,picture,connectortype_id,digitalmode_id


In [ ]:
%%sql
CREATE OR REPLACE
FUNCTION radio.radiodetails2 (INOUT radioid INT,
                              OUT radioname varchar(100))
AS $$
SELECT
	r.radio_id,
	r.radio_name
FROM
	radio.radios AS r
WHERE
	r.radio_id = radioid $$
LANGUAGE SQL;

In [6]:
%%sql
SELECT * FROM radio.radiodetails(2);

,radio_id,radio_name
0,2,UV5R


In [4]:
%%sql
drop function hello(text);
create or replace function hello (p text) returns text
language plpgsql as $$
declare
    v text;
begin
    v := 'Hello, ';
    return v || p || '!';
end;
$$;

In [13]:
%%sql
select * from hello('Slava');

,hello
0,"Hello, Slava!"


In [20]:
%%sql
CREATE TYPE airplane AS (code char(3), model text)

Exception: [42710] ОШИБКА: тип "airplane" уже существует

In [22]:
%%sql
drop function if exists plane_set2();
create or replace function plane_set2() returns setof airplane
language plpgsql as $$
declare
    v record;
begin
    for v in select * from airports
    loop
        return next row(v.airport_code, v.airport_name)::airplane;
    end loop;
end;
$$;

In [28]:
%%sql
select * from plane_set2();

,code,model
0,AAA,Anaa
1,AAC,El Arish
2,AAE,Rabah Bitat
3,AAF,Apalachicola
4,AAH,Merzbrück
...,...,...
5496,ZWA,Andapa
5497,ZWL,Wollaston Lake
5498,ZYI,Xinzhou
5499,ZYL,Osmany


In [ ]:
%%sql
CREATE OR REPLACE FUNCTION air_city(a_code text) RETURNS text
LANGUAGE plpgsql AS $$
DECLARE
    v text;
BEGIN
    SELECT city
    INTO v
    FROM airports
    WHERE airport_code = a_code;
    IF NOT FOUND THEN
        RETURN '-- Invalid airport code --';
    END IF;
    RETURN v;
END;
$$;

In [31]:
%%sql
select air_city('AAN2');

,air_city
0,-- Invalid airport code --


In [ ]:
%%sql
create or replace function hello_sql(nm text) returns text
language sql as $$
    select 'Hello ' || nm || '!';
$$;


In [32]:
%%sql
select * from hello_sql('world');

,hello_sql
0,Hello world!


# 16.5. Упражнения
**Упражнение 16.1.** Напишите на языке PL/pgSQL функцию, возвращающую все
данные, относящиеся к одному бронированию, номер которого задан параметром

In [34]:
%%sql
select * from bookings
limit 5

,book_ref,book_date,total_amount
0,2EW1SQ,2025-09-01 00:00:12.557744+00,8125.0
1,3ZY3O5,2025-09-03 00:25:35.278453+00,7500.0
2,756UAS,2025-09-10 16:24:02.978559+00,6325.0
3,GC7I6S,2025-09-03 00:26:37.085293+00,18000.0
4,RBUUIQ,2025-09-01 00:00:33.126206+00,13000.0


In [37]:
%%sql
create or replace function info_about_booking_by_book_ref(ref char(6))
    returns setof bookings
language plpgsql as $$
begin
    return query
        select * from bookings
        where book_ref = ref;
end;
$$;

In [46]:
%%sql
select * from info_about_booking_by_book_ref('3ZY3O5');

,book_ref,book_date,total_amount
0,3ZY3O5,2025-09-03 00:25:35.278453+00,7500.0


In [41]:
%%sql
drop function if exists info_about_booking_by_book_ref_table;
create or replace function info_about_booking_by_book_ref_table(ref char(6))
    returns table (
        book_ref char(6),
        book_date timestamp with time zone,
        total_amount numeric
    )
language plpgsql as $$
begin
    RETURN QUERY
    SELECT b.book_ref, b.book_date, b.total_amount
    FROM bookings b
    WHERE b.book_ref = ref;
end;
$$;

In [47]:
%%sql
select * from info_about_booking_by_book_ref_table('3ZY3O51');

,book_ref,book_date,total_amount


In [48]:
%%sql
SELECT
    kcu.table_schema || '.' || kcu.table_name AS foreign_table,     -- таблица с FK
    '→' AS relation,
    rel_tco.table_schema || '.' || rel_tco.table_name AS primary_table,  -- таблица, на которую ссылается
    string_agg(kcu.column_name, ', ' ORDER BY kcu.ordinal_position) AS fk_columns,
    kcu.constraint_name
FROM information_schema.table_constraints tco
JOIN information_schema.key_column_usage kcu
    ON tco.constraint_schema = kcu.constraint_schema
    AND tco.constraint_name = kcu.constraint_name
JOIN information_schema.referential_constraints rco
    ON tco.constraint_schema = rco.constraint_schema
    AND tco.constraint_name = rco.constraint_name
JOIN information_schema.table_constraints rel_tco
    ON rco.unique_constraint_schema = rel_tco.constraint_schema
    AND rco.unique_constraint_name = rel_tco.constraint_name
WHERE tco.constraint_type = 'FOREIGN KEY'
GROUP BY kcu.table_schema, kcu.table_name, rel_tco.table_schema, rel_tco.table_name, kcu.constraint_name
ORDER BY foreign_table;

,foreign_table,relation,primary_table,fk_columns,constraint_name
0,bookings.boarding_passes,→,bookings.segments,"ticket_no, flight_id",boarding_passes_ticket_no_flight_id_fkey
1,bookings.routes,→,bookings.airplanes_data,airplane_code,routes_airplane_code_fkey
2,bookings.routes,→,bookings.airports_data,arrival_airport,routes_arrival_airport_fkey
3,bookings.routes,→,bookings.airports_data,departure_airport,routes_departure_airport_fkey
4,bookings.seats,→,bookings.airplanes_data,airplane_code,seats_airplane_code_fkey
5,bookings.segments,→,bookings.flights,flight_id,segments_flight_id_fkey
6,bookings.segments,→,bookings.tickets,ticket_no,segments_ticket_no_fkey
7,bookings.tickets,→,bookings.bookings,book_ref,tickets_book_ref_fkey


In [49]:
%%sql
SELECT schemaname AS schema,
       tablename AS table_name
FROM pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')  -- исключаем системные
ORDER BY schemaname, tablename;

,schema,table_name
0,bookings,airplanes_data
1,bookings,airports_data
2,bookings,boarding_passes
3,bookings,bookings
4,bookings,flights
5,bookings,routes
6,bookings,seats
7,bookings,segments
8,bookings,tickets


# Учебная база от Грока
Библиотека

In [ ]:
%%sql
-- Создаём базу данных (если её нет; выполни это отдельно, если нужно)
-- CREATE DATABASE library_db;

-- Подключаемся к базе (в psql: \c library_db)

-- Создаём таблицы
CREATE TABLE IF NOT EXISTS authors (
    author_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    birth_year INTEGER
);

CREATE TABLE IF NOT EXISTS books (
    book_id SERIAL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    author_id INTEGER REFERENCES authors(author_id),
    publication_year INTEGER,
    genre VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS readers (
    reader_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    registration_date DATE DEFAULT CURRENT_DATE
);

CREATE TABLE IF NOT EXISTS borrowings (
    borrowing_id SERIAL PRIMARY KEY,
    book_id INTEGER REFERENCES books(book_id),
    reader_id INTEGER REFERENCES readers(reader_id),
    borrow_date DATE NOT NULL,
    return_date DATE
);

-- Вставляем тестовые данные
INSERT INTO authors (name, birth_year) VALUES
('J.K. Rowling', 1965),
('George R.R. Martin', 1948),
('J.R.R. Tolkien', 1892);

INSERT INTO books (title, author_id, publication_year, genre) VALUES
('Harry Potter and the Philosopher''s Stone', 1, 1997, 'Fantasy'),
('A Game of Thrones', 2, 1996, 'Fantasy'),
('The Hobbit', 3, 1937, 'Fantasy');

INSERT INTO readers (name) VALUES
('Alice Johnson'),
('Bob Smith');

INSERT INTO borrowings (book_id, reader_id, borrow_date, return_date) VALUES
(1, 1, '2025-01-01', '2025-01-15'),
(2, 2, '2025-02-01', NULL),
(3, 1, '2025-03-01', NULL);

# Первое задание: Создай простую функцию

Задание: Создай функцию get_book_count_by_genre, которая принимает жанр (строку, например 'Fantasy') и возвращает количество книг этого жанра в таблице books.

In [ ]:
%%sql
create or replace function get_book_count_by_genre(p_genre text) returns int
language plpgsql as $$
declare
    res int;
begin
    select count(*) into res
    from books where genre = p_genre;

    return res;
end;
$$;

In [54]:
%%sql
select * from get_book_count_by_genre('Fantasy');

,get_book_count_by_genre
0,3


In [ ]:
%%sql
create or replace function get_book_count_by_genre_sql(p_genre text) returns int
language sql as $$
    select count(*)
    from books where genre = p_genre;
$$;

In [55]:
%%sql
select * from get_book_count_by_genre('Fantasy');

,get_book_count_by_genre
0,3


# Второе задание

Создай функцию get_overdue_books(), которая возвращает таблицу с информацией о просроченных книгах (тех, что взяты, но не возвращены, и borrow_date прошёл больше 30 дней назад).

Она должна возвращать:
* название книги
* имя читателя
* дату взятия
* сколько дней просрочено

In [68]:
%%sql
drop function if exists get_overdue_books;
create or replace function get_overdue_books()
returns table (
    book_name varchar,
    fio varchar,
    borrow_date date,
    days_overdue int
)
language plpgsql as $$
begin
    return query
    select
        bk.title as book_name,
        r.name as fio,
        b.borrow_date,
        (current_date - b.borrow_date) as days_overdue
    from
        borrowings b join books bk
        on b.book_id = bk.book_id
        join readers r
        on b.reader_id = r.reader_id
        where return_date is null
        and (current_date - b.borrow_date) >= 30;
end;
$$;

In [66]:
%%sql
select
    bk.title as book_name,
    r.name as fio,
    b.borrow_date,
    (current_date - b.borrow_date) as days
from
    borrowings b join books bk
    on b.book_id = bk.book_id
    join readers r
    on b.reader_id = r.reader_id
    where return_date is null
    and (current_date - b.borrow_date) > 30

,book_name,fio,borrow_date,days
0,The Hobbit,Alice Johnson,2025-03-01,301
1,A Game of Thrones,Bob Smith,2025-02-01,329


In [71]:
%%sql
select * from get_overdue_books();

,book_name,fio,borrow_date,days_overdue
0,The Hobbit,Alice Johnson,2025-03-01,301
1,A Game of Thrones,Bob Smith,2025-02-01,329


In [ ]:
%%sql
drop function if exists get_overdue_books_sql;
create or replace function get_overdue_books_sql()
returns table (
    book_name varchar,
    fio varchar,
    borrow_date date,
    days_overdue int
)
language sql as $$
    select
        bk.title as book_name,
        r.name as fio,
        b.borrow_date,
        (current_date - b.borrow_date) as days_overdue
    from
        borrowings b join books bk
        on b.book_id = bk.book_id
        join readers r
        on b.reader_id = r.reader_id
        where return_date is null
        and (current_date - b.borrow_date) >= 30;
$$;

In [72]:
%%sql
select * from get_overdue_books();

,book_name,fio,borrow_date,days_overdue
0,The Hobbit,Alice Johnson,2025-03-01,301
1,A Game of Thrones,Bob Smith,2025-02-01,329


# Третье задание
Теперь перейдём к процедурам (они отличаются от функций тем, что не возвращают значение, а предназначены для выполнения действий — вставки, обновления и т.д.).

**Задание:**
Создай процедуру return_book(p_borrowing_id INTEGER), которая:

* Принимает ID записи из таблицы borrowings
* Устанавливает return_date = CURRENT_DATE
* Но только если книга ещё не возвращена (иначе — ничего не делать или выдать ошибку/предупреждение)

In [ ]:
%%sql
create or replace procedure return_book(p_borrowing_id int)
language plpgsql as $$
declare
    rd date;
begin
    select borrowings.return_date into rd
    from borrowings
    where borrowing_id = p_borrowing_id;

    if not found then
         raise exception 'borrowing not found';
    end if;

    if rd is not null then
        raise exception 'book was returned earlier';
    end if;

    update borrowings
    set return_date = current_date
    where borrowing_id = p_borrowing_id
    and return_date is null;

    raise notice 'Book successfully returned on %', CURRENT_DATE;
end;
$$;

In [74]:
%%sql
select * from borrowings;

,borrowing_id,book_id,reader_id,borrow_date,return_date
0,1,1,1,2025-01-01,2025-01-15
1,3,3,1,2025-03-01,NaN
2,2,2,2,2025-02-01,2025-12-27


In [77]:
%%sql
call return_book(33);

Exception: [P0001] ОШИБКА: borrowing not found
  Где: функция PL/pgSQL return_book(integer), строка 10, оператор RAISE

# Четвертое задание

Создай процедуру borrow_book(p_book_id INT, p_reader_id INT), которая:

* Принимает ID книги и ID читателя
* Проверяет:
    * Книга существует
    * Читатель существует
    * Книга не взята сейчас (нет активной записи в borrowings с return_date IS NULL)
* Если всё ок — вставляет новую запись в borrowings с borrow_date = CURRENT_DATE, return_date = NULL
* Если книга уже взята — бросает понятное исключение

In [78]:
%%sql
drop procedure if exists borrow_book();

create or replace procedure borrow_book(p_book_id int, p_reader_id int)
language plpgsql as $$
begin
    perform from books
    where book_id = p_book_id;

    if not found then
        raise exception 'not found book with id = %', p_book_id;
    end if;

    perform from readers
    where reader_id = p_reader_id;

    if not found then
        raise exception 'not found reader with id = %', p_reader_id;
    end if;

    perform from borrowings
    where book_id = p_book_id and return_date is null;

    if found then
        raise exception 'another read book with id = % now', p_book_id;
    end if;

    insert into borrowings (book_id, reader_id, borrow_date, return_date) VALUES
    (p_book_id, p_reader_id, current_date, null);

end;
$$;

In [83]:
%%sql
call borrow_book(1, 1);

In [85]:
%%sql
select * from borrowings

,borrowing_id,book_id,reader_id,borrow_date,return_date
0,1,1,1,2025-01-01,2025-01-15
1,2,2,2,2025-02-01,2025-12-27
2,3,3,1,2025-03-01,2025-12-27
3,4,1,1,2025-12-28,NaN


In [86]:
%%sql
call borrow_book(1, 2);

Exception: [P0001] ОШИБКА: another read book with id = 1 now
  Где: функция PL/pgSQL borrow_book(integer,integer), строка 21, оператор RAISE

In [87]:
%%sql
call borrow_book(99, 1);

Exception: [P0001] ОШИБКА: not found book with id = 99
  Где: функция PL/pgSQL borrow_book(integer,integer), строка 7, оператор RAISE

In [88]:
%%sql
call borrow_book(1, 111);

Exception: [P0001] ОШИБКА: not found reader with id = 111
  Где: функция PL/pgSQL borrow_book(integer,integer), строка 14, оператор RAISE

# Пятое задание


Создай триггер, который автоматически запрещает выдачу книги (INSERT в borrowings), если она уже взята (т.е. есть активная запись с return_date IS NULL).

* Триггер на таблице borrowings
* Тип: BEFORE INSERT
* На каждую строку (FOR EACH ROW)
* Если книга уже взята — RAISE EXCEPTION

Это будет дублировать логику из процедуры, но теперь защита будет на уровне базы — даже если кто-то вставит напрямую, без процедуры, выдача не пройдёт.

In [ ]:
%%sql
drop function if exists check_borrowed_for_trigger;

create or replace function check_borrowed_for_trigger()
returns trigger
language plpgsql as $$
begin
    perform 1 from borrowings
    where book_id = new.book_id
    and return_date is null;

    if found then
        raise exception 'another read book with id = % now', new.book_id;
    end if;

    return new;
end;
$$;


In [ ]:
%%sql
create or replace trigger book_already_borrowed_trigger
before insert on borrowings
for each row
execute function check_borrowed_for_trigger();

In [89]:
%%sql
select * from borrowings;

,borrowing_id,book_id,reader_id,borrow_date,return_date
0,1,1,1,2025-01-01,2025-01-15
1,2,2,2,2025-02-01,2025-12-27
2,3,3,1,2025-03-01,2025-12-27
3,4,1,1,2025-12-28,NaN


In [92]:
%%sql
call borrow_book(1, 1);

Exception: [P0001] ОШИБКА: another read book with id = 1 now
  Где: функция PL/pgSQL borrow_book(integer,integer), строка 21, оператор RAISE

In [93]:
%%sql
insert into borrowings (book_id, reader_id, borrow_date, return_date)
values (1, 2, current_date, null);

Exception: [P0001] ОШИБКА: another read book with id = 1 now
  Где: функция PL/pgSQL check_borrowed_for_trigger(), строка 8, оператор RAISE

# Шестое задание

Создай триггер, который при возврате книги (UPDATE таблицы borrowings, когда return_date меняется с NULL на дату) автоматически записывает в лог-таблицу информацию о возврате.
Для этого:

Создай таблицу для лога:
```postgresql
CREATE TABLE borrowing_log (
    log_id SERIAL PRIMARY KEY,
    borrowing_id INTEGER,
    book_id INTEGER,
    reader_id INTEGER,
    borrow_date DATE,
    return_date DATE,
    logged_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

Напиши триггер AFTER UPDATE ON borrowings, который срабатывает только когда OLD.return_date IS NULL AND NEW.return_date IS NOT NULL, и вставляет запись в borrowing_log.

In [ ]:
%%sql
CREATE TABLE borrowing_log (
    log_id SERIAL PRIMARY KEY,
    borrowing_id INTEGER,
    book_id INTEGER,
    reader_id INTEGER,
    borrow_date DATE,
    return_date DATE,
    logged_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

In [ ]:
%%sql
create or replace function create_log_about_return_book()
returns trigger
language plpgsql as $$
begin
    if old.return_date is null and new.return_date is not null then
        insert into borrowing_log (
            borrowing_id, book_id, reader_id, borrow_date, return_date
        )
        values (
            new.borrowing_id, new.book_id, new.reader_id, new.borrow_date,
                new.return_date
        );

        raise notice 'new log created';
    end if;

    return null;

end;
$$;

In [94]:
%%sql
create or replace trigger create_log_trigger
after update on borrowings
for each row
when (old.return_date is null and new.return_date is not null)
execute function create_log_about_return_book();

In [ ]:
%%sql
call borrow_book(2, 1);

In [99]:
%%sql
select * from borrowings;

,borrowing_id,book_id,reader_id,borrow_date,return_date
0,1,1,1,2025-01-01,2025-01-15
1,2,2,2,2025-02-01,2025-12-27
2,3,3,1,2025-03-01,2025-12-27
3,4,1,1,2025-12-28,NaN
4,6,2,1,2025-12-28,NaN


In [97]:
%%sql
call return_book(6);

In [100]:
%%sql
select * from borrowing_log;

,log_id,borrowing_id,book_id,reader_id,borrow_date,return_date,logged_at
0,1,6,2,1,2025-12-28,2025-12-28,2025-12-28 11:02:42.007257


# Седьмое задание

Напиши функцию find_overdue_readers(), которая:
* Находит всех читателей, у которых есть хотя бы одна просроченная книга (взята более 30 дней назад и не возвращена)
* Для каждого такого читателя выводит сообщение через RAISE NOTICE
    * `Reader <name> has overdue books: <количество> pcs, max overdue days: <дней>`
* Функция ничего не возвращает (VOID), просто выводит информацию.

In [ ]:
%%sql
create or replace function find_overdue_readers() returns void
language plpgsql as $$
declare
    rec record;
    reader_name varchar;
    count_books int;
    max_days int;
begin
    for rec in
        select distinct reader_id from borrowings
        where return_date is null
        and (current_date - borrow_date) > 30
    loop
        select name into reader_name from readers
        where reader_id = rec.reader_id;

        select count(*), max(current_date - borrow_date)
        into count_books, max_days
        from borrowings
        where return_date is null
        and (current_date - borrow_date) > 30
        and reader_id = rec.reader_id
        group by reader_id;

        raise notice 'Reader % has overdue books: % pcs, max overdue days: %', reader_name, count_books, max_days;
    end loop;
end;
$$;

In [108]:
%%sql
select reader_id, count(*), max(current_date - borrow_date) from borrowings
where return_date is null
group by reader_id;

,reader_id,count,max
0,1,1,0


In [116]:
%%sql
select find_overdue_readers();

,find_overdue_readers
0,NaN


In [117]:
%%sql
update borrowings
set return_date = null
where borrowing_id in (1, 2, 3)

In [118]:
%%sql
select * from borrowings

,borrowing_id,book_id,reader_id,borrow_date,return_date
0,4,1,1,2025-12-28,NaN
1,6,2,1,2025-12-28,2025-12-28
2,1,1,1,2025-01-01,NaN
3,2,2,2,2025-02-01,NaN
4,3,3,1,2025-03-01,NaN


In [119]:
%%sql
select find_overdue_readers();

,find_overdue_readers
0,NaN
